<a href="https://colab.research.google.com/github/agi2019/ppi-gci/blob/main/tutorials/0a.%20Indicator%20Pre-Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import PchipInterpolator

In [2]:
scenario = '_scenario1'
#scenario = '_scenario2'
#scenario = '_scenario3'

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/raw_data'+scenario+'/pre_raw_indicators.csv')

In [4]:
df

,seriesCode,sdg,2014,2015,2016,2017,2018,2019,2020,2021,...,2024,seriesName,bestBound,worstBound,instrumental,invert,color,2018weight,2020weight,2024weight
0,gci11_Lonline,11,2.6333,NaN,NaN,2.63330,2.6333,NaN,3.007773,NaN,...,3.423680,Laws on unauthorized online behavior,3.423680,0,1,0,#A21942,2.6333,3.7739,3.423680
1,gci11_Lforgery,11,2.6333,NaN,NaN,2.63330,2.6333,NaN,2.164725,NaN,...,2.805590,legislation in force related to computer-relat...,2.805590,0,1,0,#A21942,2.6333,3.2430,2.805590
2,gci11_Lolsafety,11,2.6333,NaN,NaN,2.63330,2.6333,NaN,2.166800,NaN,...,3.176444,Laws on online safety,3.176444,0,1,0,#A21942,2.6333,2.1668,3.176444
3,gci12_Rpdp,12,0.0000,NaN,NaN,0.49375,0.9875,NaN,1.459900,NaN,...,1.628493,Regulation(s) related to personal data protect...,1.628493,0,1,0,#FF3A21,0.9875,1.4599,1.628493
4,gci12_Rprivacy,12,0.0000,NaN,NaN,0.49375,0.9875,NaN,1.459900,NaN,...,1.183533,Regulation(s) related to privacy protection,1.183533,0,1,0,#FF3A21,0.9875,1.4599,1.183533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,gci52_Mcapacitydev,52,1.8000,NaN,NaN,1.80000,1.8000,NaN,1.989100,NaN,...,2.062319,Part of a multilateral cybersecurity agreement...,2.062319,0,1,0,#F472D0,1.8000,1.9891,2.062319
59,gci53_Mlat,53,0.0000,NaN,NaN,0.00000,0.0000,NaN,1.188100,NaN,...,3.032386,Participate in Mutual Legal Assistance Treatie...,3.032386,0,0,0,#FA6800,3.4000,1.1881,3.032386
60,gci54_PPPdomestic,54,0.0000,NaN,NaN,0.00000,0.0000,NaN,2.067300,NaN,...,3.416127,Participate in PPPs on cybersecurity with dome...,3.416127,0,1,0,#6D8764,1.7000,2.0673,3.416127
61,gci54_PPPforeign,54,0.0000,NaN,NaN,0.00000,0.0000,NaN,1.710500,NaN,...,2.304660,Participate in PPPs on cybersecurity with fore...,2.304660,0,1,0,#6D8764,1.7000,1.7105,2.304660


In [5]:
#normalized
# Define year columns to normalize and their corresponding weight columns
normalize_targets = {
    '2014': '2018weight',
    '2017': '2018weight',
    '2018': '2018weight',
    '2020': '2020weight',
    '2024': '2024weight'
}

# Copy the original dataframe to avoid modifying it in place
df_normalized = df.copy()

# Apply normalization: divide each target column by the corresponding weight column
for year_col, weight_col in normalize_targets.items():
    if year_col in df.columns and weight_col in df.columns:
        df_normalized[year_col] = df[year_col] / df[weight_col]*df['bestBound']
    else:
        print(f"Warning: column {year_col} or {weight_col} not found!")
df_normalized

,seriesCode,sdg,2014,2015,2016,2017,2018,2019,2020,2021,...,2024,seriesName,bestBound,worstBound,instrumental,invert,color,2018weight,2020weight,2024weight
0,gci11_Lonline,11,3.423680,NaN,NaN,3.423680,3.423680,NaN,2.728650,NaN,...,3.423680,Laws on unauthorized online behavior,3.423680,0,1,0,#A21942,2.6333,3.7739,3.423680
1,gci11_Lforgery,11,2.805590,NaN,NaN,2.805590,2.805590,NaN,1.872751,NaN,...,2.805590,legislation in force related to computer-relat...,2.805590,0,1,0,#A21942,2.6333,3.2430,2.805590
2,gci11_Lolsafety,11,3.176444,NaN,NaN,3.176444,3.176444,NaN,3.176444,NaN,...,3.176444,Laws on online safety,3.176444,0,1,0,#A21942,2.6333,2.1668,3.176444
3,gci12_Rpdp,12,0.000000,NaN,NaN,0.814247,1.628493,NaN,1.628493,NaN,...,1.628493,Regulation(s) related to personal data protect...,1.628493,0,1,0,#FF3A21,0.9875,1.4599,1.628493
4,gci12_Rprivacy,12,0.000000,NaN,NaN,0.591767,1.183533,NaN,1.183533,NaN,...,1.183533,Regulation(s) related to privacy protection,1.183533,0,1,0,#FF3A21,0.9875,1.4599,1.183533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,gci52_Mcapacitydev,52,2.062319,NaN,NaN,2.062319,2.062319,NaN,2.062319,NaN,...,2.062319,Part of a multilateral cybersecurity agreement...,2.062319,0,1,0,#F472D0,1.8000,1.9891,2.062319
59,gci53_Mlat,53,0.000000,NaN,NaN,0.000000,0.000000,NaN,3.032386,NaN,...,3.032386,Participate in Mutual Legal Assistance Treatie...,3.032386,0,0,0,#FA6800,3.4000,1.1881,3.032386
60,gci54_PPPdomestic,54,0.000000,NaN,NaN,0.000000,0.000000,NaN,3.416127,NaN,...,3.416127,Participate in PPPs on cybersecurity with dome...,3.416127,0,1,0,#6D8764,1.7000,2.0673,3.416127
61,gci54_PPPforeign,54,0.000000,NaN,NaN,0.000000,0.000000,NaN,2.304660,NaN,...,2.304660,Participate in PPPs on cybersecurity with fore...,2.304660,0,1,0,#6D8764,1.7000,1.7105,2.304660


In [6]:
year_cols = [str(y) for y in range(2014, 2025)]   # 2014–2024

# -------------------------------------------------------------
# 3) PCHIP INTERPOLATION FUNCTION WITH PER‑ROW BOUNDS
# -------------------------------------------------------------
def interpolate_row_pchip(row):
    """
    Fill missing year values using a monotone PCHIP spline, then
    clamp each year value to the range [0, bestBound] for this row.
    """
    # --- 3a. build PCHIP on known data ------------------------
    known_mask   = row[year_cols].notna()
    known_years  = np.array([int(y) for y in year_cols])[known_mask.values]
    known_values = row[year_cols][known_mask].values.astype(float)

    if len(known_values) >= 2:
        pchip = PchipInterpolator(known_years, known_values, extrapolate=False)
        for yr in year_cols:
            if pd.isna(row[yr]):
                row[yr] = pchip(int(yr))

    # --- 3b. clip to [0, bestBound] --------------------------
    upper = row['bestBound']
    row[year_cols] = row[year_cols].clip(lower=0, upper=upper)

    return row

df_interpolated = df.apply(interpolate_row_pchip, axis=1)
# -------------------------------------------------------------
pd.set_option('display.max_rows', None)
df_interpolated

,seriesCode,sdg,2014,2015,2016,2017,2018,2019,2020,2021,...,2024,seriesName,bestBound,worstBound,instrumental,invert,color,2018weight,2020weight,2024weight
0,gci11_Lonline,11,2.633300,2.6333,2.6333,2.6333,2.633300,2.7860137923522017,3.007773,3.1413467392387964,...,3.423680,Laws on unauthorized online behavior,3.423680,0,1,0,#A21942,2.633300e+00,3.773900e+00,3.423680
1,gci11_Lforgery,11,2.633300,2.6333,2.6333,2.6333,2.633300,2.3990123565,2.164725,2.185506374046875,...,2.805590,legislation in force related to computer-relat...,2.805590,0,1,0,#A21942,2.633300e+00,3.243000e+00,2.805590
2,gci11_Lolsafety,11,2.633300,2.6333,2.6333,2.6333,2.633300,2.40005,2.166800,2.2165221939062496,...,3.176444,Laws on online safety,3.176444,0,1,0,#A21942,2.633300e+00,2.166800e+00,3.176444
3,gci12_Rpdp,12,0.000000,0.06816077441077441,0.24604377104377104,0.49375,0.987500,1.2874597178285971,1.459900,1.5298579947020776,...,1.628493,Regulation(s) related to personal data protect...,1.628493,0,1,0,#FF3A21,9.875000e-01,1.459900e+00,1.628493
4,gci12_Rprivacy,12,0.000000,0.06816077441077441,0.24604377104377104,0.49375,0.987500,1.183533,1.183533,1.183533,...,1.183533,Regulation(s) related to privacy protection,1.183533,0,1,0,#FF3A21,9.875000e-01,1.459900e+00,1.183533
5,gci12_Rnotif,12,0.000000,0.06816077441077441,0.24604377104377104,0.49375,0.987500,1.150237,1.150237,1.150237,...,1.150237,Regulation(s) related to data breach/incident ...,1.150237,0,1,0,#FF3A21,9.875000e-01,2.220200e+00,1.150237
6,gci12_RAuditre,12,0.000000,0.06816077441077441,0.24604377104377104,0.49375,0.987500,1.056402,1.056402,1.056402,...,1.056402,Regulation(s) related to cybersecurity audit r...,1.056402,0,1,0,#FF3A21,9.875000e-01,1.522200e+00,1.056402
7,gci12_Rstandard,12,0.000000,0.06816077441077441,0.24604377104377104,0.49375,0.962567,0.9382000000000001,0.888900,0.890051039546875,...,0.962567,Regulation(s) related to cybersecurity standar...,0.962567,0,1,0,#FF3A21,9.875000e-01,8.889000e-01,0.962567
8,gci12_Rsign,12,0.000000,0.06816077441077441,0.24604377104377104,0.49375,0.987500,0.9208,0.854100,0.8634591517968749,...,1.080617,Regulation(s) related to use of digital signat...,1.080617,0,1,0,#FF3A21,9.875000e-01,8.541000e-01,1.080617
9,gci12_Rspam,12,0.000000,0.0,0.0,0.0,0.820300,0.8203,0.727200,0.7286546938749999,...,0.820300,Regulation(s) related to unsolicited communica...,0.820300,0,1,0,#FF3A21,4.200000e+00,7.272000e-01,0.820300


In [7]:
columns_to_drop = [
    '2014', '2015', '2016', '2017', '2018', '2019',
    '2018weight', '2020weight', '2024weight'
]

# Drop those columns if they exist
df_cleaned = df_interpolated.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Export to a new CSV file
df_cleaned.to_csv('raw_indicators.csv')
